In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from RecursiveFNO import RecursiveFNO
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from neuralop.models.base_model import get_model
from configmypy import ConfigPipeline, YamlConfig, ArgparseConfig


os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.manual_seed(66)
np.random.seed(66)
torch.set_default_dtype(torch.float32)
BATCH_SIZE = 1
time_steps = 38        # Total time steps per simulation
model_select = "1pde_1mse"

In [2]:
def load_checkpoint(model, optimizer, scheduler, save_dir):
    '''load model and optimizer'''
    checkpoint = torch.load(save_dir)
    model.load_state_dict(checkpoint['model_state_dict'])
    if (not optimizer is None):
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    print('Pretrained model loaded!')
    return model, optimizer, scheduler

In [3]:
config_name = "default"
pipe = ConfigPipeline(
    [
        YamlConfig("ns_pino_config.yaml", config_name="default", config_folder="config"),
    ]
)
config = pipe.read_conf()
config_name = pipe.steps[-1].config_name
model_fno = get_model(config).cuda()
model = RecursiveFNO(model_fno, time_steps).cuda()
model.eval()
model, _, _ = load_checkpoint(model, None, None, "./model/checkpoint1000_"+model_select+".pt")

Pretrained model loaded!


In [4]:
def read_data(datapath, re_list):
    vel_seq_whole = []
    for vis in re_list:
        tmp = np.load(datapath + "Re_%i.npy" % vis)
        sim_data = tmp.reshape(128, 256, 39, 3)
        rho_data_shape = (sim_data.shape[0], sim_data.shape[1], sim_data.shape[2], 1)
        rho_data = np.empty(rho_data_shape)
        rho_data[:, :, :, :] = vis * 4.0
        run_data = np.concatenate([sim_data, rho_data], axis=-1)
        # Normalization
        run_data[:, :, :, 2] /= (vis * 4.0)
        run_data[:, :, :, 3] /= 40000
        run_data = np.transpose(run_data, (2, 3, 1, 0))
        vel_seq_whole.append(run_data)
    return np.array(vel_seq_whole)

In [5]:
test_list = [20, 60, 140, 350, 550, 750, 1000]
test_seq_clipped = read_data("data/", test_list)
test_ic = torch.Tensor(test_seq_clipped[:, 0, :, :, :])
test_seq = torch.Tensor(test_seq_clipped[:, 1:, :3, :, :])
test_dataset = TensorDataset(test_ic, test_seq)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
with torch.no_grad():
    model.eval()
    for batch_id, data in enumerate(test_dataloader):
        ic, seq = data
        ic, seq = ic.cuda(), seq.cuda()
        rho = ic[:, 3, 0, 0] * 40000
        u0 = ic[:, :3, :, :]
        output = model(ic)
        output = output[0].transpose(1, 3).detach().cpu().numpy()
        fname = "data/pred_fno/%s/Re_%i.npy" % (model_select, test_list[batch_id])
        np.save(fname, output)